<a href="https://colab.research.google.com/github/boseull/tensorflow_example/blob/main/tensorflow_TF_Hub%EB%A1%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##import tf, tfhub

In [ ]:
import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds


In [ ]:
print("버전 : ", tf.__version__)
print("즉시 실행 모드", tf.executing_eagerly())
print("허브 버전", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능") #GPU로 연결해야함

버전 :  2.5.0
즉시 실행 모드 True
허브 버전 0.12.0
GPU 사용 가능


##IMDB 데이터셋 다운로드

In [ ]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews', split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

In [ ]:
len(train_data)
len(validation_data)

10000

##데이터 탐색

In [ ]:
train_examples_batch, train_lavels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_lavels_batch #라벨 데이터 0:부정 1:긍정

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 모델 구성 
####텍스트 표현 방법 / 층의 수 / 각 층에서 은닉 유닛의 수



In [ ]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True) #hub층 만듦. 
hub_layer(train_examples_batch[:3])  #hub층에 데이터 입력 후 마지막 3개 tensor 출력

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer) #사전 훈련된 모델 층 추가, 임베딩 벡터로 매핑(하나의 문장을 토큰으로 나누고 각 토큰의 임베딩을 연결하여 반환), (num_examples, embedding_dimension)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1)) #하나의 출력 sigmoid -> 0-1 실수

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

## 모델 훈련

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 70ms/step - loss: 0.6939 - accuracy: 0.5838 - val_loss: 0.6649 - val_accuracy: 0.6034
Epoch 2/20
30/30 [==============================] - 2s 67ms/step - loss: 0.6375 - accuracy: 0.6258 - val_loss: 0.6225 - val_accuracy: 0.6339
Epoch 3/20
30/30 [==============================] - 2s 67ms/step - loss: 0.5945 - accuracy: 0.6657 - val_loss: 0.5847 - val_accuracy: 0.6652
Epoch 4/20
30/30 [==============================] - 2s 66ms/step - loss: 0.5523 - accuracy: 0.6988 - val_loss: 0.5461 - val_accuracy: 0.7055
Epoch 5/20
30/30 [==============================] - 2s 67ms/step - loss: 0.5101 - accuracy: 0.7392 - val_loss: 0.5104 - val_accuracy: 0.7272
Epoch 6/20
30/30 [==============================] - 2s 68ms/step - loss: 0.4687 - accuracy: 0.7699 - val_loss: 0.4743 - val_accuracy: 0.7597
Epoch 7/20
30/30 [==============================] - 2s 67ms/step - loss: 0.4304 - accuracy: 0.7956 - val_loss: 0.4451 - val_accuracy: 0.7777
Epoch 8/20
30

##모델 평가

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s : %.3f" % (name, value))

49/49 - 2s - loss: 0.3294 - accuracy: 0.8563
loss : 0.329
accuracy : 0.856
